In [67]:
import joblib
import pandas as pd
import numpy as np
from preprocessing import preprocessing, normalization
from feature_engineering import create_feat

from utils import read_yaml

PREPROCESSING_CONFIG_PATH = "../config/preprocessing_config.yaml"
FEATURE_ENGINEERING_CONFIG_PATH = "../config/feature_engineering_config.yaml"

normalizer = joblib.load("../output/normalizer.pkl")
model = joblib.load('../output/model_name.pkl')
estimator = joblib.load('../output/best_estimator.pkl')

In [86]:
def construct_df(params, data_to_predict, file=None):
    if file == 'csv':
        df_to_predict = pd.read_csv(data_to_predict, sep = ';')
    elif file == 'excel':
        df_to_predict = pd.read_excel(data_to_predict)
    else:
        df_to_predict = pd.DataFrame(data=data_to_predict)
        df_to_predict = set_dtypes(df_to_predict, params)
        COLUMN = set(params['PREDICT_COLUMN'])
        column_in_data = set(df_to_predict.columns)
        remain_columns = list(COLUMN-column_in_data)
        df_to_predict[remain_columns] = np.NaN
    return df_to_predict

def main_predict(text, tfidf_vectorizer, model, threshold, param_preprocess, param_vec, id=0):
    df = df_constructor(text, id)
    df_preprocessed = preprocessing(df, param_preprocess)
    df_vect, _ = vectorize_tfidf(df_preprocessed, param_vec, tfidf_vectorizer)
    
    code2rel = {0: 'Non-Toxic', 1: 'Toxic'}
    df_vect = df_vect.drop(columns='id')
    proba = model.predict_proba(df_vect)[:, 1]
    predict = 1 if proba > threshold else 0
    
    return code2rel[predict], proba

In [50]:
from utils import read_yaml

PREPROCESSING_CONFIG_PATH = "../config/preprocessing_config.yaml"

params_prep = read_yaml(PREPROCESSING_CONFIG_PATH)


In [47]:
train = joblib.load('../output/x_train.pkl')
to_predict = train.iloc[[45]]
col_name = to_predict.columns
test_predict = np.array([[234, 300000, 2, 3, 1, 45, 1, -1, 0, -1, 1, -1, 3000, 4000, 245, 3560, 1200, 200, 4000, 245, 80, 3000, 1000, 100]])
df_test = pd.DataFrame(test_predict, columns=col_name)

In [64]:
feat = create_feat(df_test)
df_normalizer = preprocessing(feat, params=params_prep, state='normalizer')[0]

In [71]:
normalization(params_prep, feat, state='normal')[0]

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,PAY_AMT4,PAY_AMT5,PAY_AMT6,AgeBin,Closeness_6,Closeness_5,Closeness_4,Closeness_3,Closeness_2,Closeness_1
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
